<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# ================================================================
# MY-AI-Gizmo • FINAL WORKING LAUNCHER (Complete, single-file)
# - Robust Drive linking (symlink preferred, copy fallback)
# - Copies settings locally to avoid Drive latency when loading server
# - Unbuffered Python server launch for live logs
# ================================================================

import os
import subprocess
import shutil
import re
import time
from pathlib import Path

# Colab Drive import (must run in Colab)
try:
    from google.colab import drive
except Exception:
    drive = None

# ---------------- Configuration ----------------
REPO_ZIP = "https://github.com/gitleon8301/MY-AI-Gizmo-working/archive/refs/heads/main.zip"
WORK_DIR = Path("/content/text-generation-webui")
DRIVE_ROOT = Path("/content/drive/MyDrive/MY-AI-Gizmo")

# ---------------- Utilities ----------------
def sh(cmd, check=False):
    """Run shell command and return CompletedProcess"""
    return subprocess.run(cmd, shell=True, capture_output=True, text=True, check=check)

print("🔧 Setting up environment...")
# Fix matplotlib BEFORE any imports that care
if "MPLBACKEND" in os.environ:
    del os.environ["MPLBACKEND"]
os.environ["MPLBACKEND"] = "Agg"
print("✓ Environment configured\n")

def _ensure_drive_path(drive_path: Path):
    """Create file or directory on Drive side."""
    if drive_path.suffix:  # looks like a file
        drive_path.parent.mkdir(parents=True, exist_ok=True)
        drive_path.touch(exist_ok=True)
    else:
        drive_path.mkdir(parents=True, exist_ok=True)

def _remove_path(path: Path):
    """Remove file/dir/symlink at path."""
    try:
        if path.is_symlink():
            path.unlink()
        elif path.is_dir():
            shutil.rmtree(path)
        elif path.exists():
            path.unlink()
    except Exception:
        # best-effort removal; ignore if fails
        pass

def _create_symlink_or_fallback(src: Path, dest: Path):
    """
    Create a symlink at `dest` pointing to `src`.
    If symlink fails, fallback to copying:
      - directories -> copytree
      - files -> copy2
    """
    try:
        dest.parent.mkdir(parents=True, exist_ok=True)
        if dest.exists() or dest.is_symlink():
            _remove_path(dest)
        os.symlink(str(src), str(dest), target_is_directory=src.is_dir())
        print(f"↪ linked: {dest} -> {src}")
        return True
    except (OSError, NotImplementedError) as ex:
        # fallback: copy contents
        try:
            if src.is_dir():
                # copytree requires dest not to exist
                if dest.exists():
                    _remove_path(dest)
                shutil.copytree(src, dest)
                print(f"⤷ copied directory (fallback): {src} -> {dest}")
            else:
                dest.parent.mkdir(parents=True, exist_ok=True)
                shutil.copy2(src, dest)
                print(f"⤷ copied file (fallback): {src} -> {dest}")
            return True
        except Exception as copy_ex:
            print(f"✗ failed to link or copy {src} -> {dest}: {copy_ex}")
            return False

# ---------------- Start script ----------------
print("=" * 60)
print("🚀 MY-AI-Gizmo Final Working Setup")
print("=" * 60)

# Step 1: Mount Google Drive
print("\n📁 Step 1/5: Mounting Google Drive...")
if drive is None:
    print("⚠️  google.colab.drive not available (not running in Colab). Skipping mount.")
else:
    try:
        drive.mount("/content/drive", force_remount=False)
        print("✓ Mounted")
    except Exception as e:
        print(f"⚠️  drive.mount failed: {e}")

# Step 2: Create Drive folders
print("\n💾 Step 2/5: Creating Drive structure...")
folders = [
    "models", "loras", "training", "characters", "presets", "prompts",
    "settings", "chat-history", "instruct-history", "outputs", "images",
    "logs", "cache", "extensions", "softprompts"
]
for f in folders:
    try:
        (DRIVE_ROOT / f).mkdir(parents=True, exist_ok=True)
    except Exception as e:
        print(f"⚠️  Could not create {DRIVE_ROOT / f}: {e}")
print(f"✓ {len(folders)} folders ready")

# Step 3: Download/setup repository
print("\n📥 Step 3/5: Setting up repository...")
if not WORK_DIR.exists():
    print("  Downloading repository ZIP...")
    sh("rm -f /content/repo.zip")
    r = sh(f"wget -q -O /content/repo.zip {REPO_ZIP}")
    if r.returncode != 0:
        print("⚠️  wget failed; continuing in case repo is already present.")
    else:
        sh("unzip -q /content/repo.zip -d /content")
        # find extracted folder
        try:
            extracted = next(Path("/content").glob("MY-AI-Gizmo-working-*"))
            extracted.rename(WORK_DIR)
            print("✓ Downloaded and extracted repo")
        except StopIteration:
            print("⚠️  Could not find extracted repo after unzip.")
else:
    print("✓ Already exists")

# Ensure we operate from WORK_DIR
if WORK_DIR.exists():
    os.chdir(WORK_DIR)
else:
    print(f"⚠️  WORK_DIR {WORK_DIR} not present. Some steps may fail.")

# ---------------- Step 3b: Download 5 GGUF models ----------------
print("\n📥 Step 3b/5: Downloading default GGUF models...")

MODELS_DIR = WORK_DIR / "user_data" / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

GGUF_MODEL_URLS = [
    # LLaMA 7B mini-chat (GGUF)
    "https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.q4_K_M.gguf",
    # ChatGPT-like small GGUF
    "https://huggingface.co/TheBloke/Llama-2-7B-32K-Instruct-GGUF/resolve/main/llama-2-7b-32k-instruct.q4_K_M.gguf",
    # OSS friendly assistant
    "https://huggingface.co/TheBloke/CodeLlama-7B-GGUF/resolve/main/codellama-7b.q4_K_M.gguf",
    # Mini coder model
    "https://huggingface.co/TheBloke/CodeWizard-7B-GGUF/resolve/main/CodeWizard-7B.gguf",
    # Lightweight chat model
    "https://huggingface.co/TheBloke/Alpaca-7B-GGUF/resolve/main/alpaca-7B.gguf"
]

for url in GGUF_MODEL_URLS:
    model_name = url.split("/")[-1]
    model_path = MODELS_DIR / model_name
    if not model_path.exists():
        print(f"  Downloading {model_name} ...")
        r = sh(f"wget -q -O '{model_path}' '{url}'")
        if r.returncode == 0:
            print(f"  ✓ {model_name} downloaded to {model_path}")
        else:
            print(f"  ⚠️ Failed to download {model_name}. Check the URL!")
    else:
        print(f"  ✓ {model_name} already exists, skipping download")
# Step 4: Create symlinks (robust, cross-platform, safe fallback)
print("\n🔗 Step 4/5: Linking to Google Drive...")

links_map = [
    ("models", "models"), ("loras", "loras"), ("training", "training"),
    ("user_data/characters", "characters"), ("user_data/presets", "presets"),
    ("user_data/prompts", "prompts"), ("user_data/settings.yaml", "settings/settings.yaml"),
    ("user_data/settings.json", "settings/settings.json"), ("user_data/chat", "chat-history"),
    ("user_data/instruct", "instruct-history"), ("outputs", "outputs"),
    ("user_data/images", "images"), ("user_data/logs", "logs"),
    ("user_data/cache", "cache"), ("user_data/extensions", "extensions"),
    ("user_data/softprompts", "softprompts"),
]

for local, drive_folder in links_map:
    drive_path = DRIVE_ROOT / drive_folder
    try:
        _ensure_drive_path(drive_path)
    except Exception as e:
        print(f"⚠️  Could not ensure drive path {drive_path}: {e}")
    local_path = WORK_DIR / local

    # Remove existing local target if present
    if local_path.exists() or local_path.is_symlink():
        _remove_path(local_path)

    # Ensure parent dir for local path
    local_path.parent.mkdir(parents=True, exist_ok=True)

    # Create symlink (preferred) or fallback to copying
    _create_symlink_or_fallback(drive_path, local_path)

print("✓ All data linked to Drive (or copied as fallback)")

# ---------- QUICK FIX: ensure local settings file to avoid Drive latency ----------
print("\n⚙ Quick-fix: ensure a fast local settings file (prevents server hang on Drive reads)")
drive_settings = DRIVE_ROOT / "settings" / "settings.yaml"
local_settings = WORK_DIR / "user_data" / "settings.yaml"

local_settings.parent.mkdir(parents=True, exist_ok=True)

# If local is a symlink to Drive, remove it so we can use a local copy
if local_settings.is_symlink():
    try:
        local_settings.unlink()
    except Exception:
        pass

if drive_settings.exists():
    try:
        shutil.copy2(drive_settings, local_settings)
        size = local_settings.stat().st_size
        print(f"✓ Copied settings from Drive to local: {local_settings} ({size} bytes)")
    except Exception as e:
        print(f"⚠️  Failed to copy settings from Drive: {e}")
        try:
            local_settings.write_text("# minimal settings (auto-created)\n")
            print(f"✓ Created minimal local settings: {local_settings}")
        except Exception as e2:
            print(f"✗ Failed to create local settings file: {e2}")
else:
    # Create a minimal default settings file (keeps loading fast)
    try:
        local_settings.write_text(
            "# minimal settings (auto-created)\n"
            "listen: true\n"
            "share: true\n"
        )
        print(f"✓ Created minimal local settings: {local_settings}")
    except Exception as e:
        print(f"✗ Failed to write minimal settings: {e}")

# Show a few lines for diagnostics
try:
    print("\n--- local settings preview ---")
    with local_settings.open("r") as f:
        for i, _line in enumerate(f):
            print(_line.rstrip())
            if i >= 9:
                break
    print("--- end preview ---\n")
except Exception as e:
    print(f"⚠️  Cannot read local settings file for preview: {e}")

# Step 5: Install dependencies
print("\n⚙️  Step 5/5: Installing dependencies...")
print("  (First run: 5-10 min, subsequent runs: instant)\n")

# Ensure start script executable
try:
    sh("chmod +x start_linux.sh")
except Exception:
    pass

# Run installer with env flags as before
installer_cmd = "GPU_CHOICE=A LAUNCH_AFTER_INSTALL=FALSE INSTALL_EXTENSIONS=FALSE bash start_linux.sh"
result = sh(installer_cmd)
if result.returncode == 0:
    print("✓ Installation complete")
else:
    print("⚠️  Installation completed with warnings or non-zero exit (this can be normal).")
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print(result.stderr)

# ================================================================
# LAUNCH GRADIO UI (unbuffered Python)
# ================================================================
print("\n" + "=" * 70)
print("🌐 LAUNCHING GRADIO UI")
print("=" * 70)
print("\n⏳ Starting server - URLs will appear below...\n")
print("=" * 70 + "\n")

# Kill any stuck processes
sh("pkill -9 -f 'python server.py'")
sh("pkill -9 -f 'gradio'")
time.sleep(2)

# Set up clean environment
env = os.environ.copy()
env["MPLBACKEND"] = "Agg"
env["GRADIO_ANALYTICS_ENABLED"] = "False"
if "PYTHONPATH" in env:
    del env["PYTHONPATH"]

# Use conda python from installer if available
python_exe = WORK_DIR / "installer_files" / "env" / "bin" / "python"
if not python_exe.exists():
    python_exe = "python3"
else:
    python_exe = str(python_exe)

# Launch command (unbuffered)
cmd = f"{python_exe} -u server.py --share --listen"
print(f"🚀 Command: {cmd}\n")

proc = subprocess.Popen(
    cmd,
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    env=env,
    cwd=str(WORK_DIR)
)

local_url = None
public_url = None
shown = False
error_count = 0
error_lines = []

try:
    for line in proc.stdout:
        print(line, end="")

        # Track errors
        if any(x in line for x in ["Error", "Exception", "Traceback"]):
            error_lines.append(line)
            error_count += 1

        # Capture public Gradio URL
        if not public_url and (m := re.search(r"(https://[a-z0-9]+\.gradio\.live)", line)):
            public_url = m.group(1)

        # Capture local URL patterns
        if not local_url and (m := re.search(r"Running on local URL:\s+(http://[^\s]+)", line)):
            local_url = m.group(1)
        elif not local_url and (m := re.search(r"(http://(?:127\.0\.0\.1|0\.0\.0\.0):\d+)", line)):
            if ":5000" not in m.group(1):
                local_url = m.group(1)

        # Show success message once URLs appear
        if (local_url or public_url) and not shown:
            print("\n" + "=" * 70)
            print("✨ SUCCESS! GRADIO UI IS RUNNING! ✨")
            print("=" * 70)
            if local_url:
                print(f"\n📍 LOCAL URL  : {local_url}")
                print("   ↳ Click this in Colab")
            if public_url:
                print(f"\n🌍 PUBLIC URL : {public_url}")
                print("   ↳ Share this link!")
                print("   ↳ Use on your phone!")
            print(f"\n💾 STORAGE    : {DRIVE_ROOT}")
            print("\n" + "=" * 70)
            print("🎯 NEXT STEPS:")
            print("=" * 70)
            print("1. Click the PUBLIC URL above")
            print("2. Go to 'Model' tab in the UI")
            print("3. Download a model")
            print("4. Load the model")
            print("5. Go to 'Chat' tab")
            print("6. Start chatting!")
            print("\n" + "=" * 70)
            shown = True
            error_lines = []  # clear errors seen before success
except Exception as e:
    print(f"\n✗ Exception while reading server output: {e}")

# If process ended without showing a UI
if proc.poll() is not None and not shown:
    print("\n" + "=" * 70)
    print("⚠️  SERVER STOPPED WITHOUT LAUNCHING")
    print("=" * 70)
    if error_lines:
        print("\n🔍 Error detected (last lines):")
        for line in error_lines[-30:]:
            print(line, end="")
    print("\n" + "=" * 70)
    print("🔧 TRY THIS:")
    print("1. Restart Colab runtime (Runtime → Restart runtime)")
    print("2. Re-run this entire script")
    print("3. Use the diagnostic commands below to inspect settings & permissions")
    print("=" * 70)
    # Diagnostic hints
    print("\nDiagnostics:")
    print(" - ls -l /content/text-generation-webui/user_data/settings.yaml")
    print(" - wc -c /content/text-generation-webui/user_data/settings.yaml")
    print(" - head -n 40 /content/text-generation-webui/user_data/settings.yaml")
    print(" - ls -l /content/drive/MyDrive/MY-AI-Gizmo/settings")
    print("=" * 70)

print("\n✓ Script finished")
print(f"💾 Your data is safe in: {DRIVE_ROOT}")
